# HV19.10 Guess what

With the new (v3) fixed binary, this was easy - I suspect there would be a harder "proper" solution, maybe?

Running the binary asks for an input:

In [1]:
!./guess3

Your input: 

With some input, we get an error:

In [4]:
!echo foo | ./guess3

nooooh. try harder!


When giving an empty input, we get a bash error:

In [5]:
!echo | ./guess3

./guess3: line 4: [: =: unary operator expected
nooooh. try harder!


## Interesting but irrelevant analysis

With [ltrace](https://www.ltrace.org/) we can find out that it does some weird stuff with environment variables (which change every time) and executes itself. Probably to make it harder to attach a debugger?

In [6]:
!echo | ltrace -e execvp+getenv+putenv ./guess3

guess3->getenv("x2e7e802a0a5b594b")              = nil
guess3->putenv("x2e7e802a0a5b594b=33502560908609"...) = 0
guess3->execvp(0x555ba2cba09a, 0x555ba2e78530, 32, 0x7ffd9033ef48 <no return ...>
--- Called exec() ---
--- Called exec() ---
guess3->getenv("x2e7e802a0a5b594b")              = "3350256090860968267 1"
guess3->execvp(0x5575de99809a, 0x5575e00b12a0, 32, 0x7ffc43710278 <no return ...>
--- Called exec() ---
./guess3: line 4: [: =: unary operator expected
nooooh. try harder!
+++ exited (status 0) +++


In [8]:
!echo | ltrace -e execvp+getenv+putenv ./guess3

guess3->getenv("xbb69d14a79547d5b")              = nil
guess3->putenv("xbb69d14a79547d5b=13504555075440"...) = 0
guess3->execvp(0x563178b4509a, 0x56317a284540, 32, 0x7fffaaa4f178 <no return ...>
--- Called exec() ---
--- Called exec() ---
guess3->getenv("xbb69d14a79547d5b")              = "13504555075440508251 1"
guess3->execvp(0x56332b84b09a, 0x56332c6002a0, 32, 0x7fff989c4e18 <no return ...>
--- Called exec() ---
./guess3: line 4: [: =: unary operator expected
nooooh. try harder!
+++ exited (status 0) +++


`strace` reveals that it calls `getpid()` before doing that. With [LD_PRELOAD](https://jvns.ca/blog/2014/11/27/ld-preload-is-super-fun-and-easy/) and the [set_pid](https://github.com/AdamSimpson/set_pid) preload library we can influence which PID it gets, which makes those numbers stable:

In [10]:
!echo | LD_PRELOAD="$PWD/set_pid/libset_pid.so" OLCF_PID=42 ltrace -e execvp+getenv+putenv-@libset_pid.so ./guess3

guess3->getenv("x874892971c6f3796")              = nil
guess3->putenv("x874892971c6f3796=97482025711582"...) = 0
guess3->execvp(0x555cd3cfe09a, 0x555cd448e540, 32, 0x7ffd2a3423c8 <no return ...>
--- Called exec() ---
--- Called exec() ---
guess3->getenv("x874892971c6f3796")              = "9748202571158206358 1"
guess3->execvp(0x55601a06809a, 0x55601bea62a0, 32, 0x7ffe15eea918 <no return ...>
--- Called exec() ---
./guess3: line 4: [: =: unary operator expected
nooooh. try harder!
+++ exited (status 0) +++


In [11]:
!echo | LD_PRELOAD="$PWD/set_pid/libset_pid.so" OLCF_PID=42 ltrace -e execvp+getenv+putenv-@libset_pid.so ./guess3

guess3->getenv("x874892971c6f3796")              = nil
guess3->putenv("x874892971c6f3796=97482025711582"...) = 0
guess3->execvp(0x55f4c0b5d09a, 0x55f4c0fc4540, 32, 0x7ffc4fba8da8 <no return ...>
--- Called exec() ---
--- Called exec() ---
guess3->getenv("x874892971c6f3796")              = "9748202571158206358 1"
guess3->execvp(0x55fd94dd609a, 0x55fd95db52a0, 32, 0x7ffcb6a90648 <no return ...>
--- Called exec() ---
./guess3: line 4: [: =: unary operator expected
nooooh. try harder!
+++ exited (status 0) +++


If we now set that environment variable to the correct value, it doesn't re-execute itself:

In [13]:
!echo | LD_PRELOAD="$PWD/set_pid/libset_pid.so" OLCF_PID=42 x874892971c6f3796="9748202571158206358 1" ltrace -e execvp+getenv+putenv-@libset_pid.so ./guess3

guess3->getenv("x874892971c6f3796")              = "9748202571158206358 1"
guess3->execvp(0x5578d5e8a09a, 0x5578d702a2a0, 32, 0x7fffa0a86578 <no return ...>
--- Called exec() ---
./guess3: line 4: [: =: unary operator expected
nooooh. try harder!
+++ exited (status 0) +++


We can also set the " 1" at the end to something else... However, 0 does the same, 2 always seems to re-execute... With 3 we get an error:

In [15]:
!echo | LD_PRELOAD="$PWD/set_pid/libset_pid.so" OLCF_PID=42 x874892971c6f3796="9748202571158206358 3" ltrace -e execvp+getenv+putenv-@libset_pid.so ./guess3

guess3->getenv("x874892971c6f3796")              = "9748202571158206358 3"
./guess3: abnormal behavior!
+++ exited (status 1) +++


## Solution

We already found out that the binary calls bash somehow. To do so, it'll need to pass a bash script, either as file or as string. Let's try to find out more via [strace](https://strace.io):

In [16]:
!echo | strace -e execve ./guess3

execve("./guess3", ["./guess3"], 0x7ffef9990ab0 /* 72 vars */) = 0
execve("/bin/bash", ["./guess3", "-c", "exec './guess3' \"$@\"", "./guess3"], 0x55e65735f2d0 /* 73 vars */) = 0
execve("/home/florian/proj/hackinglab/2019_hackvent/10/guess3", ["./guess3"], 0x5621657283c0 /* 73 vars */) = 0
execve("/bin/bash", ["./guess3", "-c", "                                "..., "./guess3"], 0x7ffd1d452898 /* 72 vars */) = 0
./guess3: line 4: [: =: unary operator expected
nooooh. try harder!
+++ exited with 0 +++


Looks like there is some whitespace there - let's tell strace to show us more.

In [17]:
!echo | strace -e execve -s 8192 ./guess3

execve("./guess3", ["./guess3"], 0x7fff344629d0 /* 72 vars */) = 0
execve("/bin/bash", ["./guess3", "-c", "exec './guess3' \"$@\"", "./guess3"], 0x556f24c712d0 /* 73 vars */) = 0
execve("/home/florian/proj/hackinglab/2019_hackvent/10/guess3", ["./guess3"], 0x56322ef503c0 /* 73 vars */) = 0
execve("/bin/bash", ["./guess3", "-c", "                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

If we print that string, we see the script and flag embedded into it:

In [18]:
print("#!/bin/bash\n\nread -p \"Your input: \" input\n\nif [ $input = \"HV19{Sh3ll_0bfuscat10n_1s_fut1l3}\" ] \nthen\n  echo \"success\"\nelse \n  echo \"nooooh. try harder!\"\nfi\n\n")

#!/bin/bash

read -p "Your input: " input

if [ $input = "HV19{Sh3ll_0bfuscat10n_1s_fut1l3}" ] 
then
  echo "success"
else 
  echo "nooooh. try harder!"
fi




## Solution 2

We can tell bash to do the equivalent of `set -x` (tracing of various builtin calls) via environment variables:

In [19]:
!echo blah | env SHELLOPTS=xtrace ./guess3

+ exec ./guess3
+ read -p 'Your input: ' input
+ '[' blah = 'HV19{Sh3ll_0bfuscat10n_1s_fut1l3}' ']'
+ echo 'nooooh. try harder!'
nooooh. try harder!


## Solution 3

Use `LD_PRELOAD` and [execvehack](https://github.com/gonoph/execvhack) to log the arguments:

In [21]:
!echo blah | env LD_PRELOAD=execvhack/execvhack.so ./guess3

Loading hack.
/bin/bash: argv[0]: [[./guess3]]
/bin/bash: argv[1]: [[-c]]
/bin/bash: argv[2]: [[exec './guess3' "$@"]]
/bin/bash: argv[3]: [[./guess3]]
Loading hack.
/home/florian/proj/hackinglab/2019_hackvent/10/guess3: argv[0]: [[./guess3]]
Loading hack.
/bin/bash: argv[0]: [[./guess3]]
/bin/bash: argv[1]: [[-c]]
/bin/bash: argv[2]: [[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## Solution 4

Use the Linux Audit framework to log the arguments.

Set up audit:
    
```
sudo auditctl -a exit,always -F path=/bin/bash -F arch=x86_64 -S execve
sudo systemctl start auditd
```

Run `./guess3` and find this in `/var/log/audit/audit.log`:
    
```

type=EXECVE msg=audit(1576014591.504:1146):  a2[1]=2020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202023212F62696E2F626173680A0A72656164202D702022596F757220696E7075743A202220696E7075740A0A6966205B2024696E707574203D2022485631397B5368336C6C5F306266757363617431306E5F31735F667574316C337D22205D200A7468656E0A20206563686F202273756363657373220A656C7365200A20206563686F20226E6F6F6F6F682E207472792068617264657221220A66690A0A a3="./guess3"
```

Using a bit of Python, we can get the script again:


In [24]:
import binascii

data = "2020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202023212F62696E2F626173680A0A72656164202D702022596F757220696E7075743A202220696E7075740A0A6966205B2024696E707574203D2022485631397B5368336C6C5F306266757363617431306E5F31735F667574316C337D22205D200A7468656E0A20206563686F202273756363657373220A656C7365200A20206563686F20226E6F6F6F6F682E207472792068617264657221220A66690A0A"
print(binascii.unhexlify(data).strip().decode('ascii'))

#!/bin/bash

read -p "Your input: " input

if [ $input = "HV19{Sh3ll_0bfuscat10n_1s_fut1l3}" ] 
then
  echo "success"
else 
  echo "nooooh. try harder!"
fi


## Bonus

The variable expansion in `if [ $input = "HV..." ]` is [improperly quoted](https://mywiki.wooledge.org/Quotes).

With a specially crafted input, we can trick the condition to read `[ ! a = "HV.." ]` so it prints "success" despite not having the flag:
    

In [1]:
!echo "! a" | ./guess3

success
